In [1]:
# PROJETO DE CONEXÃO COM PLATAFORMA MONDAY PARA EXTRAÇÃO DE ATIVIDADES E ROTINAS DIARIAS
# Importando Biblioteca para Extração de API
import requests

# Importando Pandas para tratamento e leitura dos dados
import pandas as pd
import datetime

# Acessar componentes do SISTEAMA OPERCIONAL
import os
import asyncio

# Instalar o pacote (python-telegram-bot)
from telegram import Bot

In [3]:
# Configurado Report para Envio de Informações em caso de erro por Telegram!!!

# define o token do seu bot do Telegram
TOKEN = 'INSERIR_TOKEN_DO_SEU_BOT_TELEGRAM'
# inicializa o objeto bot
bot = Bot(token=TOKEN)
# Incluir código do grupo
GRUPO = 'INSERIR_CÓDIGO_DO_GRUPO_QUE_DESEJA_ENVIAR_O_ALERTA'

In [15]:
# Key de acesso a API
apiKey = "Inseri_API_DO_SEU_ADM_MONDAY"

# URL de Acesso a API > Monday
apiUrl = "https://api.monday.com/v2"

# Variavel para Parametro do Requeset Liberar acesso
headers = {"Authorization" : apiKey}

# Query de linguagem GRAPHQL para acesso de API (CONSULTA PODE VARIAR AGORA COM SUA NOVA VERSÃO > E DADOS DIVERGEM DEPENDEDO DO TIPO DE AREA ESCOLHIDA OU CRIADA NO MONDAY)
query2 = '{boards() { name id description items { id name column_values{title id type text } } } }'
data = {'query' : query2}
print('Realizado a solicitação de conexão com a API')

# Realizado conexão com API
r = requests.post(url=apiUrl, json=data, headers=headers) # make request

Realizado a solicitação de conexão com a API


In [16]:

#Condição de validação se conexão com API foi um Sucesso
if r.status_code == 200:
    print('Conexão com a API foi um Sucesso!!!')
    # Usando o json_normalize para transformar os dados em um DataFrame
    df = pd.json_normalize(r.json(), record_path=['data', 'boards', 'items','column_values'], 
    meta=[['data', 'boards', 'items','name'],['data', 'boards', 'name'],['data', 'boards', 'name', 'id'] ])
else:
    print("Falha na Atualização de Relatório, Erro: Status de conexão com a API {r.status_code}")
    # Mensagem que você deseja enviar
    mensagem = f"Falha na Atualização de Relatório DP, Erro: Status de conexão com a API {r.status_code}"
    # Enviar a mensagem desejada
    # Cria uma função assicronica para Enviar_mensagem():
    async def enviar_foto():
        await bot.send_message(chat_id=GRUPO, text=mensagem)
    # chama a função assíncrona usando asyncio
    loop = asyncio.get_event_loop()
    loop.run_until_complete(enviar_foto()) 
    exit()

Conexão com a API foi um Sucesso!!!


In [17]:
# Organiza as colunas para realizar a alteração de linhas para colunas!!!
df = df[['data.boards.name.id','data.boards.name','data.boards.items.name', 'title', 'text']]

# Realiza a troca dos nomes das colunas para melhor entedimento da Base
df.rename(columns={'data.boards.name.id': 'ID',
                   'data.boards.name': 'Quadro',
                   'data.boards.items.name': 'Elemento'},
          inplace = True)

In [18]:
# Use a função pivot para transformar os dados de linhas para colunas mantendo as colunas (ID,Quadro e Elemento)
df = df.pivot(index=['ID', 'Quadro', 'Elemento'], columns='title', values='text').reset_index()

In [19]:
# Armazena informação da data atual para acrescenter ao nome do arquivo Historico 
data_atual = str(datetime.date.today())

In [20]:
# Remover as duplicadas entre o arquivo historico e Datframe novo extraido pela API
df.drop_duplicates(subset=['ID'], ignore_index=True, inplace= True)
 
# Realizado a alteração do Tipo da Coluna Data
df['Tempo Estimado'] = pd.to_numeric(df['Tempo Estimado'])
# Criando uma função para criar uma nova coluna com tempo em segundo
def tempo_segundo(Col):
 Col = (Col * 60)
 Col = (Col * 60)
 return Col

In [21]:
# Aplicando a função para a nova coluna para obter os dados em Segundos
df['Tempo_Estimado_Seg'] = df['Tempo Estimado'].apply(tempo_segundo)

# Criando Coluna com a data Atual
def data_hora_atual(Col):
 Col = str(datetime.datetime.today())
 return Col
df['Data_Atual'] = ''
df['Data_Atual'] = df['Data_Atual'].apply(data_hora_atual)

In [22]:
df.head()

title,ID,Quadro,Elemento,Data,Empresa,Prioridade,Responsável,Rotina,Status,Tempo Estimado,Área,Tempo_Estimado_Seg,Data_Atual
0,5520656387,Atividades_Diarias,Renovação de Estoque,2023-11-12,LL,ALTA,Ezequiel Cleydenilson,,Não Iniciada,1,MARKETING,3600,2023-11-15 22:46:38.778735
1,5520656392,Atividades_Diarias,Melhoria de Memoria,2023-11-18,ADV,BAIXA,Ezequiel Cleydenilson,,Não Iniciada,10,ADMINISTRATIVO,36000,2023-11-15 22:46:38.778735
2,5520656399,Atividades_Diarias,Alteração de Dados,2023-11-12,LL,BAIXA,Ezequiel Cleydenilson,,Concluido,4,CS,14400,2023-11-15 22:46:38.778735
3,5520656410,Atividades_Diarias,Testes de Equipamentos,2023-11-17,ADV,MÉDIA,Ezequiel Cleydenilson,,Em andamento,5,MIS,18000,2023-11-15 22:46:38.778735
4,5520656414,Atividades_Diarias,Atualização de Maquinas,2023-11-17,SYS,ALTA,Ezequiel Cleydenilson,,Em Aprovação,6,DADOS,21600,2023-11-15 22:46:38.778735
